In [2]:
import pandas as pd
import numpy as np

class RectangluarGridPath:
    
    # creating a pandas dataframe to be filled later
    def __init__(self, m , n):
        init_tuple = [(i, j) for i in range(m + 1) for j in range(n + 1)]
        self.path_lst = pd.DataFrame(data= { 'x': [i[0] for i in init_tuple], 'y':[i[1] for i in init_tuple]  }, 
                                     columns=['x', 'y', 'probability', 'distance'] )
        self.path_lst.set_index(['x', 'y'])
        self.m = m
        self.n = n

    def getPathEndPoints(self):
        return self.path_lst
    
    def __getPathIndex(self, x, y):
        idx = self.path_lst.index[(self.path_lst['x'] == x) & (self.path_lst['y'] == y)][0] 
        return idx
    
    def getProbAndDistance(self, x, y):
        idx = self.__getPathIndex(x, y)
        return self.path_lst['probability'][idx], self.path_lst['distance'][idx]
    
    def setToPath(self, x, y, prob, dist):
        idx = self.__getPathIndex(x, y)
        self.path_lst.at[idx,'probability'] = prob
        self.path_lst.at[idx , 'distance'] = dist
    
    # in order to achive better accuracy I change the distance equation to abs(x*n - y*m ),
    #but when I calculate the statisticn parameters(i.e. mean and std) I devide the result by m*n
    def calcPointDistanc(self, x, y):
        return abs((x * self.n)- (y*self.m))
        
    def addToPath(self, x , y):
        if x == 0 and y == 0:
            self.setToPath(0, 0, np.array([1]), np.array([0]))
            
        elif x == 0:
            [probs, dists] = self.getProbAndDistance(x, y-1)
            dist = self.calcPointDistanc(x, y)
            dists = np.array([dist])
            self.setToPath(x, y, probs, dists)
            
        elif y == 0: 
            [probs, dists] = self.getProbAndDistance(x -1, y)
            dist = self.calcPointDistanc(x, y)
            dists = np.array([dist])
            self.setToPath(x, y, probs, dists)
            
        else:
            [probs1, dists1] = self.getProbAndDistance(x - 1, y)
            [probs2, dists2] = self.getProbAndDistance(x, y - 1)
            dist = self.calcPointDistanc(x, y)
            dists = np.append(dists1, dists2)
            probs = np.append(probs1, probs2) * 1/2
            
            # The folloing lines are used to shrink the size of probability and distance lists for boosting the performance:
            
            # Grouping repeated distance
            for item in set(dists):
                con, = np.where(dists == item)
                new_prob  = np.sum(probs[con])
                probs = np.round(np.append(np.array(new_prob), np.delete(probs, con)), 8)
                dists = np.append(np.array(item), np.delete(dists,con))

                # Removing distance with zero probability
                con, = np.where(probs == 0)
                new_prob  = np.sum(probs[con])
                probs = np.delete(probs, con)
                dists =  np.delete(dists,con)

                #Shrink the probability and distance lists
                condition, = np.where(dists <= dist)
                new_prob  = np.sum(probs[condition])

                if len(condition) > 0:
                    #print(x, '  ', y)
                    new_probs = np.append(np.array(new_prob), np.delete(probs, condition)) 
                    new_dists = np.append(np.array(dist), np.delete(dists,condition)) 
                    self.setToPath(x, y, new_probs, new_dists)
                else:
                    self.setToPath(x, y, probs, dists)
    
    def calcMean(self, x, y):
        [probs, dists] = self.getProbAndDistance(x, y)
        return np.round(np.sum(dists * probs)/ (self.m * self.n), 10)
    
    def calcStd(self, x, y):
        [probs, dists] = self.getProbAndDistance(x, y)
        return np.round(np.std(dists * probs)/ (self.m * self.n), 10)
    
    def ConditionalProb_Distance(self, x, y, treshold_dist, dist):
        scaled_treshold_dist = treshold_dist * self.m * self.n
        scaled_dist = dist * self.m * self.n
        
        [probs, dists] = self.getProbAndDistance(x, y)
        threshold_filter, = np.where(dists > scaled_treshold_dist)
        dist_filter, =  np.where(dists > scaled_dist)
        
        treshold_prob = np.sum(probs[threshold_filter])
        dist_prob = np.sum(probs[dist_filter])
        
        return np.round(dist_prob/treshold_prob, 10)
    
    def statisticFactors(self, x, y, treshold_dist, dist):
        mean = self.calcMean(x, y)
        std = self.calcStd(x, y)
        con_dist = self.ConditionalProb_Distance(x, y, treshold_dist, dist)
        print('\n', 'mean : ', mean , '\n', 'std : ', std, '\n', 'conditional_distance : ' , con_dist)
    


In [53]:
m = 23
n = 31

path = RectangluarGridPath(m, n)
for i in range(m+1):
    for j in range(n+1):
        path.addToPath(i,j)

path.statisticFactors(x = m, y = n, treshold_dist = .2, dist = 0.6)



 mean :  0.3532883329 
 std :  0.0020721955 
 conditional_distance :  0.0631263541
